In [1]:
import sys
sys.path.append('../scared_lbc')

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import cupy as _cu
from tqdm.auto import tqdm
import numpy as np
import math

In [4]:
import scared_cu

In [6]:
reduction = scared_cu.lbc.modop.Reduction_0Q(257, 'int16')

model = scared_cu.models.SignedHammingWeight(expected_dtype='int16')
# model = scared_cu.models.ShiftRight(4)
# model = ShiftRightAndMask(6, 6)
# model = BinHammingWeight(expected_dtype='int32')
# model = scared_cu.models.Mask(6)
# model = scared_cu.Value()

partitions = _cu.arange(9)


N = 15000
sigma = 1
conv = 3000

n = 32 #don't touch
n_step = 32

neg_trick = False

prng_off = False

incomplete = 0

In [7]:
def kw_wrapper(ths, reduction, model, partitions, step=1000, conv_step=1000, neg_trick=False, subkeys=None, early_stop=True, pp=False, accept_shift=False, sa=2):

    if incomplete == 0:
        selection_function = scared_cu.lbc.selection_functions.BaseMul(reduction=reduction, words=subkeys)
    else:
        selection_function = scared_cu.lbc.selection_functions.BaseMulIncomplete(reduction=reduction, words=subkeys)

    kw_attack = scared_cu.KWAttack(selection_function=selection_function,
                                               model=model,
                                               discriminant=scared_cu.maxabs,
                                               convergence_step=conv_step,
                                               partitions=partitions)

    mean = ths.samples.mean(axis=0)
    _n = ths.samples.shape[1] // 2
    pp_ = scared_cu.preprocesses.high_order.CenteredProduct(mean=mean, frame_1=range(0,_n), frame_2=range(_n,2*_n), mode='same')

    for i in tqdm(range(0, len(ths), step)):
        if pp:
            container = scared_cu.Container(ths[i:i + step], preprocesses=[pp_])
        else:
            container = scared_cu.Container(ths[i:i + step])
        kw_attack.run(container)

        if early_stop:
            succs = scared_cu._utils.succ_ratio(ths.s[0, subkeys], reduction.q, conv, kw_attack, accept_shift=accept_shift, sa=sa, incomplete=incomplete!=0)
            if succs[-1] == 1.0:
                break

    return kw_attack

In [8]:
ths = scared_cu.lbc.simulated_traces.collect_traces_basemult(N=N, n=n, sigma=sigma, reduction=reduction, incomplete=incomplete,
                              const_seed=True, two_step=False, seed=0, prng_off=prng_off, leak_reduction=True)

print(ths)

  0%|          | 0/15000 [00:00<?, ?it/s]

Trace Header Set:
Name.............: RAM Format THS
Reader...........: RAM reader with 15000 traces. Samples shape: (15000, 64) and metadatas: ['c', 's', 's0', 's1']
c................: int16
s................: int16
s0...............: int16
s1...............: int16



In [9]:
succs = None


for n_i in tqdm(range(0, n, n_step)):
    subkeys = _cu.arange(n_i, n_i + n_step)
    assert ((ths.s % reduction.q) == ((ths.s0 + ths.s1) % reduction.q)).all()
    kw = kw_wrapper(ths[:N], reduction, model, partitions, step=conv, conv_step=conv, neg_trick=neg_trick, subkeys=subkeys, early_stop=False, pp=not prng_off)
    succs_i_ = scared_cu._utils.succ_ratio(ths.s[0, subkeys], reduction.q, conv, kw, incomplete=incomplete!=0)
    succs_i = _cu.zeros(N//conv)
    for i in range(len(succs_i_)):
        succs_i[i] = succs_i_[i]
    for i in range(len(succs_i_), len(succs_i)):
        succs_i[i] = succs_i_[-1]
    if succs is None:
        succs = _cu.array(succs_i)
    else:
        succs += _cu.array(succs_i)

    for x,y in zip(range(conv,N+1,conv),succs):
        print(f'({math.log(x)/math.log(10)},{y})',end='')
    # if n_i != (n - n_step):
    #     del kw
    print()
    for i in range(n_i, n_i + n_step):
        print(f'{ths.s[0, i]}-{kw.scores[:,i-n_i].argsort()[-1]}', end='\t')
    print()

succs /= (n//n_step)
assert (kw.data == (-1)).sum() == 0


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

(3.4771212547196617,0.625)(3.778151250383643,0.84375)(3.9542425094393248,0.9375)(4.079181246047624,0.96875)(4.176091259055681,1.0)
183-183	184-184	42-42	254-254	47-47	60-60	193-193	203-203	242-242	6-6	48-48	221-221	234-234	242-242	170-170	97-97	220-220	142-142	125-125	33-33	67-67	67-67	201-201	157-157	86-86	133-133	171-171	42-42	240-240	7-7	235-235	105-105	


In [10]:
for x,y in zip(range(conv,N+1,conv),succs):
    print(f'({math.log(x)/math.log(10)},{y})',end='')

(3.4771212547196617,0.625)(3.778151250383643,0.84375)(3.9542425094393248,0.9375)(4.079181246047624,0.96875)(4.176091259055681,1.0)